In [58]:
import os
from moviepy.editor import AudioFileClip, TextClip, CompositeVideoClip, ColorClip
from moviepy.video.tools.subtitles import SubtitlesClip
from IPython.display import Video
from open_whisper import preprocess
import subprocess
import numpy as np
import pandas as pd

In [2]:
preprocess.download_audio(
    video_id="eh77AUKedyM",
    output_dir="/home/ubuntu/open_whisper/data/audio/eh77AUKedyM",
    ext="m4a",
)

In [3]:
preprocess.download_transcript(
    video_id="eh77AUKedyM",
    lang_code="en-US",
    output_dir="/home/ubuntu/open_whisper/scripts/notebooks",
    sub_format="srt",
)

In [3]:
command = ["yt-dlp", "--list-formats", "https://www.youtube.com/watch?v=eh77AUKedyM"]
subprocess.run(command)

[youtube] Extracting URL: https://www.youtube.com/watch?v=eh77AUKedyM
[youtube] eh77AUKedyM: Downloading webpage
[youtube] eh77AUKedyM: Downloading ios player API JSON
[youtube] eh77AUKedyM: Downloading android player API JSON
[youtube] eh77AUKedyM: Downloading m3u8 information
[info] Available formats for eh77AUKedyM:
ID  EXT   RESOLUTION FPS CH |   FILESIZE    TBR PROTO | VCODEC           VBR ACODEC      ABR ASR MORE INFO
-------------------------------------------------------------------------------------------------------------------------
sb2 mhtml 48x27        1    |                   mhtml | images                                   storyboard
sb1 mhtml 80x45        1    |                   mhtml | images                                   storyboard
sb0 mhtml 160x90       1    |                   mhtml | images                                   storyboard
233 mp4   audio only        |                   m3u8  | audio only           unknown             [en] Default
234 mp4   audio 

CompletedProcess(args=['yt-dlp', '--list-formats', 'https://www.youtube.com/watch?v=eh77AUKedyM'], returncode=0)

In [6]:
command = ["yt-dlp", "--list-subs", "https://www.youtube.com/watch?v=jDwJawAsDbg"]
subprocess.run(command, check=True)

[youtube] Extracting URL: https://www.youtube.com/watch?v=jDwJawAsDbg
[youtube] jDwJawAsDbg: Downloading webpage
[youtube] jDwJawAsDbg: Downloading ios player API JSON
[youtube] jDwJawAsDbg: Downloading android player API JSON
[youtube] jDwJawAsDbg: Downloading m3u8 information
[info] Available automatic captions for jDwJawAsDbg:
Language      Name                                                Formats
en-GB                                                             vtt
af-en-GB      Afrikaans from English (United Kingdom)             vtt, ttml, srv3, srv2, srv1, json3
ak-en-GB      Akan from English (United Kingdom)                  vtt, ttml, srv3, srv2, srv1, json3
sq-en-GB      Albanian from English (United Kingdom)              vtt, ttml, srv3, srv2, srv1, json3
am-en-GB      Amharic from English (United Kingdom)               vtt, ttml, srv3, srv2, srv1, json3
ar-en-GB      Arabic from English (United Kingdom)                vtt, ttml, srv3, srv2, srv1, json3
hy-en-GB      Armen

CompletedProcess(args=['yt-dlp', '--list-subs', 'https://www.youtube.com/watch?v=jDwJawAsDbg'], returncode=0)

### FFMpeg - doesn't work so slay


In [23]:
transcript_file = "/home/ubuntu/open_whisper/data/transcripts/eh77AUKedyM/segments/00:00:01.501_00:00:30.071.vtt"
audio_file = "/home/ubuntu/open_whisper/data/audio/eh77AUKedyM/segments/00:00:01.501_00:00:30.071.m4a"

In [25]:
command = [
    "ffmpeg",
    "-f",
    "lavfi",
    "-i",
    "color=c=black:s=1280x720:r=30",
    "-i",
    f"{audio_file}",
    "-vf",
    f"subtitles={transcript_file}",
    "-c:v",
    "libx264",
    "-c:a",
    "aac",
    "/home/ubuntu/open_whisper/data/sanity-check/output.mp4",
]
subprocess.run(command)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

CompletedProcess(args=['ffmpeg', '-f', 'lavfi', '-i', 'color=c=black:s=1280x720:r=30', '-i', '/home/ubuntu/open_whisper/data/audio/eh77AUKedyM/segments/00:00:01.501_00:00:30.071.m4a', '-vf', 'subtitles=/home/ubuntu/open_whisper/data/transcripts/eh77AUKedyM/segments/00:00:01.501_00:00:30.071.vtt', '-c:v', 'libx264', '-c:a', 'aac', '/home/ubuntu/open_whisper/data/sanity-check/output.mp4'], returncode=1)

### MoviePy - works! so slay


In [59]:
def generate_video(audio_file, transcript_file, output_file):
    audio = AudioFileClip(audio_file)
    generator = lambda text: TextClip(
        text, font="Helvetica", fontsize=24, color="white"
    )

    if open(transcript_file).read() == "":
        print("Silent segment")
        video = ColorClip(size=(800, 420), color=(0, 0, 0), duration=audio.duration)
        video = video.set_audio(audio)
        video.write_videofile(output_file, fps=24)
    else:
        subtitles = SubtitlesClip(transcript_file, generator).set_position(
            ("center", "bottom")
        )
        video = CompositeVideoClip([subtitles], size=(800, 420))
        video = video.set_audio(audio)
        video.write_videofile(output_file, fps=24)


def view_video(output_file):
    return Video(output_file, width=800, height=420)

#### Sanity-Checking for tiny-en training


In [8]:
id_list = os.listdir("/home/ubuntu/open_whisper/data/sanity-check/transcripts")
id_list

['DqSJCL8zHnM',
 'EOttef29t9Y',
 'cuHScB5zGRA',
 'YHul1eMhCzA',
 'm_Wi_QncGxo',
 'i35mA0Q-mbg',
 'DU1f6D7D3po',
 'GkXOtldEFWk',
 'KPE5W8MuRF8',
 '2CWmrpsN41E']

In [39]:
id_segments = {id: [] for id in id_list}
for id in id_list:
    audio_dir = f"/home/ubuntu/open_whisper/data/sanity-check/audio/{id}/segments"
    transcript_dir = (
        f"/home/ubuntu/open_whisper/data/sanity-check/transcripts/{id}/segments"
    )

    audio_segments = sorted(
        [
            os.path.join(audio_dir, segment)
            for segment in os.listdir(
                f"/home/ubuntu/open_whisper/data/sanity-check/audio/{id}/segments"
            )
        ]
    )

    transcript_segments = sorted(
        [
            os.path.join(transcript_dir, segment)
            for segment in os.listdir(
                f"/home/ubuntu/open_whisper/data/sanity-check/transcripts/{id}/segments"
            )
        ]
    )

    os.makedirs(f"/home/ubuntu/open_whisper/data/segment-check/{id}", exist_ok=True)

    output_file_list = [
        f"/home/ubuntu/open_whisper/data/segment-check/{id}/{i + 1}.mp4"
        for i, segment in enumerate(audio_segments)
    ]

    id_segments[id] = list(zip(audio_segments, transcript_segments, output_file_list))

In [88]:
id_segments["KPE5W8MuRF8"]

[('/home/ubuntu/open_whisper/data/sanity-check/audio/KPE5W8MuRF8/segments/00:00:03.369_00:00:31.564.m4a',
  '/home/ubuntu/open_whisper/data/sanity-check/transcripts/KPE5W8MuRF8/segments/00:00:03.369_00:00:31.564.srt',
  '/home/ubuntu/open_whisper/data/segment-check/KPE5W8MuRF8/1.mp4'),
 ('/home/ubuntu/open_whisper/data/sanity-check/audio/KPE5W8MuRF8/segments/00:00:31.564_00:00:58.090.m4a',
  '/home/ubuntu/open_whisper/data/sanity-check/transcripts/KPE5W8MuRF8/segments/00:00:31.564_00:00:58.090.srt',
  '/home/ubuntu/open_whisper/data/segment-check/KPE5W8MuRF8/2.mp4'),
 ('/home/ubuntu/open_whisper/data/sanity-check/audio/KPE5W8MuRF8/segments/00:00:58.090_00:01:26.319.m4a',
  '/home/ubuntu/open_whisper/data/sanity-check/transcripts/KPE5W8MuRF8/segments/00:00:58.090_00:01:26.319.srt',
  '/home/ubuntu/open_whisper/data/segment-check/KPE5W8MuRF8/3.mp4'),
 ('/home/ubuntu/open_whisper/data/sanity-check/audio/KPE5W8MuRF8/segments/00:01:26.319_00:01:55.915.m4a',
  '/home/ubuntu/open_whisper/data

In [92]:
generate_video(*id_segments["KPE5W8MuRF8"][1])

                                                                       



                                                         
                                                                    

                                                               


chunk:   4%|▍         | 29/650 [1:11:45<25:36:43, 148.48s/it, now=None]





                                                                       



                                                           
                                                                    

                                                                 


chunk:   4%|▍         | 29/650 [1:11:45<25:36:43, 148.48s/it, now=None]







Moviepy - Building video /home/ubuntu/open_whisper/data/segment-check/KPE5W8MuRF8/2.mp4.
MoviePy - Writing audio in 2TEMP_MPY_wvf_snd.mp3






















                                                                       



                                                           
                                                                    

                                                                 


chunk:   4%|▍         | 29/650 [1:11:46<25:36:50, 148.49s/it, now=None]





                                                                       



                                                           
                                                                    

                                                                 


chunk:   4%|▍         | 29/650 [1:11:46<25:36:50, 148.49s/it, now=None]







MoviePy - Done.
Moviepy - Writing video /home/ubuntu/open_whisper/data/segment-check/KPE5W8MuRF8/2.mp4























































































                                                                       



                                                           
                                                                    

                                                                 


chunk:   4%|▍         | 29/650 [1:11:48<25:37:40, 148.57s/it, now=None]





                                                                       



                                                           
                                                                    

                                                                 


chunk:   4%|▍         | 29/650 [1:11:48<25:37:40, 148.57s/it, now=None]







Moviepy - Done !
Moviepy - video ready /home/ubuntu/open_whisper/data/segment-check/KPE5W8MuRF8/2.mp4


In [93]:
view_video(id_segments["KPE5W8MuRF8"][1][2])

#### Sanity Check


In [36]:
transcript_file = "/home/ubuntu/open_whisper/data/transcripts/eh77AUKedyM/segments/00:00:01.501_00:00:30.071.srt"
audio_file = "/home/ubuntu/open_whisper/data/audio/eh77AUKedyM/segments/00:00:01.501_00:00:30.071.m4a"

In [37]:
audio_clip = AudioFileClip(audio_file)
audio_clip.duration

28.58

In [38]:
start_time = 0
generator = lambda text: TextClip(text, font="Helvetica", fontsize=24, color="white")
subtitles = SubtitlesClip(transcript_file, generator).set_position(("center", "bottom"))

In [39]:
video = CompositeVideoClip([subtitles], size=(800, 420)).set_audio(audio_clip)

In [40]:
video.write_videofile("/home/ubuntu/open_whisper/data/sanity-check/test.mp4", fps=24)

Moviepy - Building video /home/ubuntu/open_whisper/data/sanity-check/test.mp4.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /home/ubuntu/open_whisper/data/sanity-check/test.mp4



Moviepy - Done !
Moviepy - video ready /home/ubuntu/open_whisper/data/sanity-check/test.mp4


In [41]:
Video("/home/ubuntu/open_whisper/data/segment-check/test.mp4", width=800, height=420)